In [94]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import sys
from IPython.display import Image, display
import copy
import random
import pandas as pd
import numpy as np

In [100]:
%cd ..
%cd /gdrive/My\ Drive/Colab\ Notebooks/AI/finalProject
!pwd

/gdrive/My Drive/Colab Notebooks/AI
/gdrive/My Drive/Colab Notebooks/AI/finalProject
/gdrive/My Drive/Colab Notebooks/AI/finalProject


In [0]:
# !wget http://archive.ics.uci.edu/ml/machine-learning-databases/00507/wisdm-dataset.zip

In [0]:
# !ls -la
# !unzip wisdm-dataset.zip
# !ls -la

In [121]:
#Take raw data from directories
import glob 

frames = []
%cd /gdrive/My\ Drive/Colab\ Notebooks/AI/finalProject/wisdm-dataset/arff_files/watch/accel
files = glob.glob('*.arff')
files.sort()
frames.append(createDfs(files))

%cd /gdrive/My\ Drive/Colab\ Notebooks/AI/finalProject/wisdm-dataset/arff_files/watch/gyro
files = glob.glob('*.arff')
files.sort()
frames.append(createDfs(files))

%cd /gdrive/My\ Drive/Colab\ Notebooks/AI/finalProject/wisdm-dataset/arff_files/phone/accel
files = glob.glob('*.arff')
files.sort()
frames.append(createDfs(files))

%cd /gdrive/My\ Drive/Colab\ Notebooks/AI/finalProject/wisdm-dataset/arff_files/phone/gyro
files = glob.glob('*.arff')
files.sort()
frames.append(createDfs(files))

from google.colab import files
saveFrame(frames)












/gdrive/My Drive/Colab Notebooks/AI/finalProject/wisdm-dataset/arff_files/watch/accel
/gdrive/My Drive/Colab Notebooks/AI/finalProject/wisdm-dataset/arff_files/watch/gyro
/gdrive/My Drive/Colab Notebooks/AI/finalProject/wisdm-dataset/arff_files/phone/accel
/gdrive/My Drive/Colab Notebooks/AI/finalProject/wisdm-dataset/arff_files/phone/gyro


In [0]:
#Download frames

def saveFrame(frames):
  for i, frame in enumerate(frames):
    for j, sample in enumerate(frame):
      if(i == 0):
        name = 'watch_accel_sample_' + str(j) +'.csv'
        sample.to_csv(name, index=False)
        files.download(name)
      elif (i == 1):
        name = 'watch_gyro_sample' + str(j)+'.csv'
        sample.to_csv(name, index=False)
        files.download(name)
      elif (i == 2):
        name = 'phone_accel_sample_' + str(j)+'.csv'
        sample.to_csv(name, index=False)
        files.download(name)
      elif (i == 3):
        name = 'phone_gyro_sample' + str(j)+'.csv'
        sample.to_csv(name, index=False)
        files.download(name)

In [113]:
!pip install liac-arff

In [0]:
def printAll(value):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(value)

In [0]:
def dataList(sample):
    #Create dataFrame
    data = arff.load(open(sample)) 
    dfi = pd.DataFrame(data['data'])
    return dfi

In [0]:
def addColumns(dfr, features):
  dfr.columns = features
  return dfr

In [0]:
def createDfs(files):
  #From arff to data frame
  import arff
  #Map all samples in dataframes
  samplesDf = list(map(dataList, files))
  #Extract columns' names
  data = arff.load(open(files[0]))
  atr = data["attributes"]
  columnas = [atr[k][0] for k in range(len(atr))]
  samplesDf = [addColumns(m, columnas) for m in samplesDf]
  return samplesDf

In [0]:
def activityType(activity : str) -> int:
    noHand = ['A', 'B', 'C', 'M', 'E']
    eating = ['H', 'I', 'J', 'K', 'L']
    if (activity in noHand):
      return 0
    elif (activity in eating):
      return 1
    else:
      return 2


In [0]:
def mapActivitiesTemp(activities):
  mapped = activities
  map(activityType, mapped)

In [0]:
def mapActivities(activities):
  noHand = ['A', 'B', 'C', 'M', 'E']
  eating = ['H', 'I', 'J', 'K', 'L']
  mapped = activities
  for i in range(0,len(activities)):
    if (activities[i] in noHand):
      mapped[i] = 0
    elif (activities[i] in eating):
      mapped[i] = 1
    else:
      mapped[i] = 2
  return mapped

In [0]:
#Directly related features are wrapped in intervals on a list
def selectFeatures(features) -> list:
  labels = copy.deepcopy(features)
  labels = labels.tolist()
  intervals = []
  atts = []
  intervals.append([2, 31]) 
  intervals.append([32, 34]) # {X,Y,Z}AVG 
  intervals.append([35, 37]) # {X,Y,Z}PEAK 
  intervals.append([38, 40]) # {X,Y,Z}ABSOLDEV 
  intervals.append([41, 43]) # {X,Y,Z}STANDDEV 
  intervals.append([44, 46]) # {X,Y,Z}VAR* 
  intervals.append([47, 85]) # FCC
  intervals.append([86, 88]) # COS
  intervals.append([89, 91]) # COR
  intervals.append([92, 92]) # resultant
  selected = []
  nFeatures = random.randint(0, 1)
  for i in range (0, nFeatures + 1):
    selected.append(i)

  for element in selected:
    ini, fin = intervals[element]
    if (ini != 92):
      for i in range(ini, fin + 1):
        atts.append(features[i])
    else:
      atts.append(features[92])
  return atts


In [0]:
#Kmeans clustering
def scoreK(values, ground, clusters):
  from sklearn.cluster import KMeans
  from sklearn import metrics
  # en n_clusters ponemos el número de actividades = 18
  kmeans = KMeans(n_clusters=clusters).fit(values)
  predicho = kmeans.predict(values)

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)
  
  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  
  score = (rand, mutual)
  return score

In [0]:
#Agglomerative clustering
def scoreAgglo(values, ground, clusters):
  from sklearn.cluster import AgglomerativeClustering
  from sklearn import metrics
  
  # en n_clusters ponemos el número de actividades = 18
  aglo = AgglomerativeClustering(n_clusters= clusters).fit(values)
  predicho = aglo.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)

  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)

  score = (rand, mutual)
  return score

In [0]:
#'' clustering
def scoreSpecs(values, ground, clusters):

  from sklearn.cluster import AgglomerativeClustering
  from sklearn import metrics

  # en n_clusters ponemos el número de actividades = 18
  aglo = AgglomerativeClustering(n_clusters= clusters).fit(values)
  predicho = aglo.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  
  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  score.append(rand)
  score.append(mutual)
  return score

In [0]:
#Calculate adjusted_rand_score and mutual_info_score for each sample and each clustering method
def calcScore(sample):
  
  #Set values to evalute clustering score
  ground_truth = sample["ACTIVITY"].values
  lookupTable, ground_truth = np.unique(ground_truth, return_inverse=True)
  features = selectFeatures(sample.columns)
  X = sample[features].values

  #Evaluate clustering method

  #Kmeans
  temRand,temMutual = scoreK(X, ground_truth, 18)
  scoresK = (temRand, temMutual)

  #Agglomerative
  temRand,temMutual = scoreAgglo(X, ground_truth, 18)
  scoresAgglo = (temRand, temMutual)
  scores = [scoresK, scoresAgglo]
  return scores

In [91]:
#rank is a list that contains 50 list (one per sample), a those 50 lists have a list for clustering method.
# The most nested list contain the values for the two calculated scores
rank = list(map(calcScore, samplesDf[:3]))
scoreFrame = pd.DataFrame(rank)
cluster = ['Kmeans', 'Agglomerative']
scoreFrame.columns = cluster
printAll(scoreFrame)

,Kmeans,Agglomerative
0,"(0.5779, 0.7518)","(0.5799, 0.7585)"
1,"(0.5403, 0.71)","(0.5813, 0.7469)"
2,"(0.3316, 0.5177)","(0.3422, 0.5258)"


In [0]:
# Wrap activities in three categories as suggested in dataset description 
# seleccionar = ["ACTIVITY"]
# X = allData[seleccionar].values
# activityType = copy.deepcopy(X)
# activityType = mapActivities(activityType)
# allData.insert(0, "actType", activityType, True)